In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import time


%load_ext autoreload
%autoreload 2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from helpers_create_data import *
from implementations import *

In [3]:
#We import the data here
from helpers import load_csv_data
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("./dataset", sub_sample=False)

In [4]:
#Here we extract the wanted features, and take care of the nans
X_train, Y_train, X_val, Y_val, X_test = make_data('./dataset/x_train.csv', './dataset/x_test.csv', x_train, x_test, y_train, replace=False, onehotecode = False)

In [5]:
#We undersample by dropping half of the data points in the majortity class
#We oversample by multiplying the minority class s.t. the number of data points is half the number of datapoints of majority class)
X_train_balanced, Y_train_balanced = undersampling_oversampling(X_train, Y_train, ratio_majority=0.5, ratio_majority_to_minority=2)

In [6]:
#Gamma = 0.05 was giving us good results while converging in acceptable time
gamma = 0.05
max_iter = 10000

#We add a column of  ones before training
tx_train = np.c_[np.ones((Y_train_balanced.shape[0], 1)), X_train_balanced]
#Reshape y_train form (#points,1) to (#points,) in order to use the implemented logistic regression function
yx_train = Y_train_balanced.reshape(-1)
#Create a new w in order to match the number of sected feature and has shape (1 + #features, )
w_reg = np.zeros((tx_train.shape[1], 1)).reshape(-1)
#Train model (-> our train set) using logistic regression
w, loss = logistic_regression(yx_train, tx_train, w_reg, max_iter, gamma)

In [7]:
print(w)

[-1.56581718e+00  3.26037763e-02  2.71478500e-01  3.04059991e-01
 -1.68470451e-01  2.09970840e-01 -9.96740449e-02  1.00003362e-01
 -4.56104921e-02 -5.42285509e-02  5.69149390e-03  7.55699438e-02
  5.27262183e-01  2.77692224e-02  1.92281179e-02  1.17660907e-01
  3.95530408e-01  8.39177315e-01  6.61353076e-04 -8.31970944e-02]


In [8]:
#Here we compute the accuracy and the f1 score for the validation set
tx_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]
#y_pred_test are the predicted labels for the validation set
y_pred_test = prediction(tx_val, w)
Y_val = Y_val.reshape(-1)
print('Accuracy:', compute_accuracy(Y_val, y_pred_test))
print('F1: ', f1(y_pred_test, Y_val))

Accuracy: 86.74305915338432
F1:  0.4116851501217203


In [18]:
#Here we actually make the prediction for the test set
tx_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
y_pred = prediction(tx_test, w)
y_pred[y_pred == 0] = -1

In [20]:
from helpers import create_csv_submission
create_csv_submission(test_ids, y_pred, "Submission_06.10.2024_16_12")